In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
data = pd.read_csv("./pqt.csv")
data.head()

,doctor,Gender,Age,st
0,ent,m,50,29
1,paediatrics,m,11,10
2,paediatrics,f,20,14
3,ophthalmology,f,50,11
4,o&g,f,34,27


In [3]:
len(data)

369

In [4]:
data['doctor'].unique()

array(['ent', 'paediatrics', 'ophthalmology', 'o&g',
       'general practitioner', 'psychiatry', 'surgical',
       'multiple diagnosis & others', 'oncology'], dtype=object)

In [5]:
print("min age: ", min(data['Age']))
print("max age: ", max(data['Age']))

min age:  2
max age:  62


In [6]:
train, test = np.split(data, [int(len(data) * 0.8)])

In [7]:
print("training data: ", len(train))
print("test data: ", len(test))

training data:  295
test data:  74


In [8]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels = df.pop('st')
    df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [9]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)

C:\Users\ASUS\AppData\Local\Temp/ipykernel_14996/3143059055.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}


In [10]:
[(train_features, label_batch)] = train_ds.take(1)
print('Features:', list(train_features.keys()))
print('A batch of ages:', train_features['Age'])
print('A batch of st:', label_batch )

Features: ['doctor', 'Gender', 'Age', 'st']
A batch of ages: tf.Tensor(
[[34]
 [11]
 [34]
 [20]
 [34]], shape=(5, 1), dtype=int64)
A batch of st: tf.Tensor([45 23 20 10 35], shape=(5,), dtype=int64)


In [11]:
def normalization_layer(name, dataset):
    # Create a Normalization layer for the feature.
    normalizer = tf.keras.layers.Normalization(axis=None)
    feature_ds = dataset.map(lambda x, y: x[name])
    normalizer.adapt(feature_ds)
    return normalizer

In [12]:
st_col = train_features['st']
layer = normalization_layer('st', train_ds)
layer(st_col)

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[ 1.0701298 ],
       [-0.30134383],
       [-0.48836297],
       [-1.11176   ],
       [ 0.4467327 ]], dtype=float32)>

In [13]:
def category_encoding_layer(name, dataset, dtype, max_tokens=None):
    # make a layer, turn strings to integer indices
    if dtype == 'string':
        index = tf.keras.layers.StringLookup(max_tokens=max_tokens)
    # else, make a layer, turn integer values to integer indices
    else:
        index = tf.keras.layers.IntegerLookup(max_tokens=max_tokens)

    feature_ds = dataset.map(lambda x, y: x[name])
    index.adapt(feature_ds)

    # Encode the integer indices
    encoder = tf.keras.layers.CategoryEncoding(num_tokens=index.vocabulary_size())

    # Using multi-hot encoding to the indices with lambda function
    return lambda feature: encoder(index(feature)) 

In [14]:
test_doctor_col = train_features['doctor']
test_doctor_layer = category_encoding_layer(name='doctor',
                                            dataset=train_ds,
                                            dtype='string')
test_doctor_layer(test_doctor_col)

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], dtype=float32)>

In [15]:
test_age_col = train_features['Age']
test_age_layer = category_encoding_layer(name='Age',
                                         dataset=train_ds,
                                         dtype='int64',
                                         max_tokens=5)
test_age_layer(test_age_col)

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)>

In [16]:
batch_size = 1
train_ds = df_to_dataset(train, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

C:\Users\ASUS\AppData\Local\Temp/ipykernel_14996/3143059055.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}


In [17]:
all_inputs = []
encoded_features = []

In [18]:
age_col = tf.keras.Input(shape=(1,), name='Age', dtype='int64')

encoding_layer = category_encoding_layer(name='Age',
                                         dataset=train_ds,
                                         dtype='int64',
                                         max_tokens=5)
encoded_age_col = encoding_layer(age_col)
all_inputs.append(age_col)
encoded_features.append(encoded_age_col)

In [19]:
categorical_cols = ['doctor', 'Gender']

for col in categorical_cols:
    categorical_col = tf.keras.Input(shape=(1,), name=col, dtype='string')
    encoding_layer = category_encoding_layer(name=col,
                                             dataset=train_ds,
                                             dtype='string',
                                             max_tokens=11)
    encoded_categorical_col = encoding_layer(categorical_col)
    all_inputs.append(categorical_col)
    encoded_features.append(encoded_categorical_col)

In [20]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(64, activation='relu')(all_features)
x = tf.keras.layers.Dense(32, activation='relu')(x)
output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(all_inputs, output)

model.compile(optimizer='adam',
              loss='mae',
              metrics=["mae"])

In [21]:
model.fit(train_ds, epochs=10)

Epoch 1/10


C:\Users\ASUS\anaconda3\lib\site-packages\keras\engine\functional.py:559: UserWarning: Input dict contained keys ['st'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


295/295 [==============================] - 1s 1ms/step - loss: 16.4863 - mae: 16.4863
Epoch 2/10
295/295 [==============================] - 0s 1ms/step - loss: 8.8638 - mae: 8.8638A: 0s - loss: 9.5672 - mae:
Epoch 3/10
295/295 [==============================] - 0s 1ms/step - loss: 5.8338 - mae: 5.8338
Epoch 4/10
295/295 [==============================] - 0s 1ms/step - loss: 4.6282 - mae: 4.6282
Epoch 5/10
295/295 [==============================] - 0s 1ms/step - loss: 4.2399 - mae: 4.2399
Epoch 6/10
295/295 [==============================] - 0s 1ms/step - loss: 4.3013 - mae: 4.3013
Epoch 7/10
295/295 [==============================] - 0s 1ms/step - loss: 4.1266 - mae: 4.1266
Epoch 8/10
295/295 [==============================] - 0s 1ms/step - loss: 4.1037 - mae: 4.1037
Epoch 9/10
295/295 [==============================] - 0s 1ms/step - loss: 3.9493 - mae: 3.9493
Epoch 10/10
295/295 [==============================] - 0s 1ms/step - loss: 3.9204 - mae: 3.9204


In [22]:
model.evaluate(test_ds)

74/74 [==============================] - 0s 1ms/step - loss: 4.1067 - mae: 4.1067


[4.106741905212402, 4.106741905212402]

In [23]:
sample = {
    'doctor': 'paediatrics',
    'Gender': 'm',
    'Age': 11
}

In [24]:
sample = {
    'doctor': '0&g',
    'Gender': 'f',
    'Age': 34
}

In [25]:
input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
predictions = model.predict(input_dict)
#prob = tf.nn.sigmoid(predictions[0])
print(predictions)

[[25.793827]]
